In [66]:
import os
import requests
from dotenv import load_dotenv
from odata import ODataService
from authlib.integrations.requests_client import OAuth2Session

In [67]:
load_dotenv(".env")

True

In [68]:
BRIGHT_MLS_AUTH_URL = os.environ.get("BRIGHT_MLS_AUTH_URL", "")
BRIGHT_MLS_API_URL = os.environ.get("BRIGHT_MLS_API_URL", "")
BRIGHT_MLS_CLIENT_ID = os.environ.get("BRIGHT_MLS_CLIENT_ID", "")
BRIGHT_MLS_CLIENT_SECRET = os.environ.get("BRIGHT_MLS_CLIENT_SECRET", "")

In [82]:
def get_oauth_token():
    session = OAuth2Session(BRIGHT_MLS_CLIENT_ID, BRIGHT_MLS_CLIENT_SECRET, scope="clientcred")
    # print("---session", session)

    token = session.fetch_token(BRIGHT_MLS_AUTH_URL, grant_type="client_credentials")
    # print("---token", token)

    return token["access_token"]

def get_service():
    session = requests.Session()
    session.headers.update(
        {
            "User-Agent": "Bright WebAPI/1.0",
            "Authorization": f"Bearer {get_oauth_token()}",
        }
    )

    return ODataService(
        BRIGHT_MLS_API_URL,
        session=session,
        reflect_entities=True,
        reflect_output_package="backend.apps.brightmls.entities_cache",
    )

In [83]:
service = get_service()

Output()

Output()

Output()

Output()

Output()

Output()

Output()

In [72]:
entities = service.entities
entities

{'BrightMedia': odata.metadata.EntitySetBrightMedia,
 'BrightMembers': odata.metadata.EntitySetBrightMembers,
 'BrightOffices': odata.metadata.EntitySetBrightOffices,
 'BrightOpenHouses': odata.metadata.EntitySetBrightOpenHouses,
 'BrightProperties': odata.metadata.EntitySetBrightProperties,
 'BuildingName': odata.metadata.EntitySetBuildingName,
 'City': odata.metadata.EntitySetCity,
 'CityZipCode': odata.metadata.EntitySetCityZipCode,
 'Deletion': odata.metadata.EntitySetDeletion,
 'GreenVerification': odata.metadata.EntitySetGreenVerification,
 'History': odata.metadata.EntitySetHistory,
 'PartyPermissions': odata.metadata.EntitySetPartyPermissions,
 'PropertyArea': odata.metadata.EntitySetPropertyArea,
 'Room': odata.metadata.EntitySetRoom,
 'School': odata.metadata.EntitySetSchool,
 'SchoolDistrict': odata.metadata.EntitySetSchoolDistrict,
 'SchoolElementary': odata.metadata.EntitySetSchoolElementary,
 'SchoolHigh': odata.metadata.EntitySetSchoolHigh,
 'SchoolMiddle': odata.metadat

In [182]:

entity_class = service.entities['BuilderModel']
entity_class.__dict__

KeyError: 'BuilderModel'

In [105]:

fields = []
for field_name, field_obj in entity_class.__dict__.items():
    if hasattr(field_obj, 'type'):
        fields.append({"Name": field_name, "Type": field_obj.type})    

# Display all fields and their types
for field in fields:
    print(f"Field Name: {field['Name']}, Field Type: {field['Type']}")    

In [64]:




# Team = service.entities["Team"]
# TeamMember = service.entities["TeamMember"]
# BrightMembers = service.entities["BrightMembers"]
city_entity = service.entities["City"]

In [63]:
# Try to determine the number of cities in the database
# query = service.query(city_entity)
# query = query.count()
# query = query.limit(10)

ODataError: HTTP 501 | 501 | No processor for interface 'CountEntityCollectionProcessor' registered. | None

In [62]:
offset = 0
limit = 1000

while True:
    print(f'--- offset:{offset} ---')
    query = service.query(city_entity)
    query = query.offset(offset).limit(limit)
    
    cities = query.all()
    print(f"--- fetched cities: {len(cities)}")
    
    if not cities:
        break
    
    offset += limit
    
    for entity in cities[:5]:
        print(entity.CtyCityCounty)
        
    print('---')
    
    if offset > 2000:
        break

# print(len(cities))
# for entity in cities[:5]:
#     print(service.values(entity))

--- offset:0 ---
--- fetched cities: 1000
DEKALB-IN
PIKE-KY
STODDARD-MO
OKANOGAN-WA
LYON-NV
---
--- offset:1000 ---
--- fetched cities: 1000
MOUNTRAIL-ND
HARFORD-MD
HAMPTON-SC
MCLENNAN-TX
ALBEMARLE-VA
---
--- offset:2000 ---
--- fetched cities: 1000
HARPER-KS
PLATTE-MO
NOME-AK
PIKE-KY
HAMPDEN-MA
---


In [20]:

# get all teams
query = service.query(Team)
teams = query.all()

In [35]:
team_instances = []
for team in teams[8:10]:
    # print(team)
    # print(service.values(team))
    print(team.TeamName)
    print(team.TeamLeadMemberKey)
    if team.TeamLeadMemberKey:
        print(service.query(BrightMembers).filter(BrightMembers.MemberKey == team.TeamLeadMemberKey).one().MemberFullName)
    print()

Let's Move Crew
121136934864
Katharine J Hopkins

Mika New team for approval 2 Testing New Team Name Mika New team for approval 2
None



In [113]:
# From cache

import backend.apps.brightmls.entities_cache as cache
from odata import property

In [165]:
entity_name = "City"
property_entity = getattr(cache, entity_name)()
type(property_entity.__odata__)
# property_entity.__odata__.describe()
property_entity.__odata__.properties

[('CtyCityCounty', <Property(CtyCityCounty)>),
 ('CtyCityKey', <Property(CtyCityKey)>),
 ('CtyCityName', <Property(CtyCityName)>),
 ('CtyCityTowhnship', <Property(CtyCityTowhnship)>),
 ('CtyCityType', <Property(CtyCityType)>),
 ('CtyCountyState', <Property(CtyCountyState)>),
 ('CtyModificationTimestamp', <Property(CtyModificationTimestamp)>)]

In [166]:
django_fields_mapping = {
    "IntegerProperty": "models.IntegerField",
    "StringProperty": "models.CharField",
    "DatetimeProperty": "models.DateTimeField",
    "BooleanProperty": "models.BooleanField",
    "FloatProperty": "models.FloatField",
    "DecimalProperty": "models.DecimalField",
    "UUIDProperty": "models.UUIDField",
    "NavigationProperty": "models.CharField",
}

fields = []
for prop_name, prop_obj in property_entity.__odata__.properties:
    # print(prop_obj)
    # print(prop_obj.__class__)
    # print(prop_obj.name)
    # print(prop_obj.primary_key)
    
    fields.append({
        "name": prop_obj.name, 
        "class": django_fields_mapping[prop_obj.__class__.__name__],
        "primary_key": prop_obj.primary_key,
    })

fields        

[{'name': 'CtyCityCounty', 'class': 'models.CharField', 'primary_key': False},
 {'name': 'CtyCityKey', 'class': 'models.IntegerField', 'primary_key': True},
 {'name': 'CtyCityName', 'class': 'models.CharField', 'primary_key': False},
 {'name': 'CtyCityTowhnship',
  'class': 'models.IntegerField',
  'primary_key': False},
 {'name': 'CtyCityType', 'class': 'models.CharField', 'primary_key': False},
 {'name': 'CtyCountyState', 'class': 'models.CharField', 'primary_key': False},
 {'name': 'CtyModificationTimestamp',
  'class': 'models.DateTimeField',
  'primary_key': False}]

In [170]:
print(f"class {entity_name}(BaseModel):")
for field in fields:
    primay_key = "primary_key=True " if field["primary_key"] else ""
    
    print(f"    {field['name']} = {field['class']}({primay_key})")

class City(BaseModel):
    CtyCityCounty = models.CharField()
    CtyCityKey = models.IntegerField(primary_key=True )
    CtyCityName = models.CharField()
    CtyCityTowhnship = models.IntegerField()
    CtyCityType = models.CharField()
    CtyCountyState = models.CharField()
    CtyModificationTimestamp = models.DateTimeField()


In [206]:
import json

# Load the specification file
def load_specification(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Mapping of specification types to Django field types (without hardcoded max_length and null)
type_mapping = {
    "string": "models.CharField",
    "integer": "models.IntegerField",
    "boolean": "models.BooleanField",
    "double": "models.FloatField",
    "datetime": "models.DateTimeField",
    "date": "models.DateField",
    "url": "models.URLField",
    "text": "models.TextField",
    "array": "models.JSONField",
    # Add more type mappings as necessary
}

# Generate Django model code from the specification data
def generate_django_model_code(spec_data):
    infos = []
    model_code = ""
    for entity_name, entity_info in spec_data["components"]["schemas"].items():
        # Clean the entity name by removing any prefix like "BrightMLS.OData.bright."
        class_name = entity_name.split(".")[-1]
        
        # print("class_name", class_name)
        
        # if class_name == "BrightProperty":
        #     continue
        # 
        # Start the model class
        model_code += f"class {class_name}(BaseModel):\n"
        
        # infos += [str(field_info) for field_name, field_info in entity_info.get("properties", {}).items()]
        # # print unique list values
        # for v in set(infos):
        #     print(v)
        #     
        # print()
        # print()
        
            
        
        # Loop through the fields and generate Django fields based on type
        for field_name, field_info in entity_info.get("properties", {}).items():
            # print("field_name", field_name, "field_info", field_info)

            # determine a primary key
            is_primary_key = field_name == f"{class_name}Key"

            # Determine max_length if specified
            max_length = field_info.get("maxLength", None)

            # Determine null based on the nullable attribute
            is_null = field_info.get("nullable", None)

            field_type = field_info.get("type", "string") # default type is string for each anyOf

            field_format = field_info.get("format", None)

            # Some corrections for specific field formats
            if field_type == 'integer' and field_format == 'int64':
                field_type = 'bigint'
            elif field_type == "string" and field_format == 'date':
                field_type = 'date'
            elif field_type == "string" and field_format == 'date-time':
                field_type = 'datetime'

            # some correction for specific field names
            if field_type == "string" and "URL" in field_name:
                field_type = "url"
                max_length = 4000
                
            if field_type == "string" and max_length is None:
                max_length = 255

            django_field_type = type_mapping.get(field_type, "models.CharField")

            field_options = []
            if max_length is not None:
                field_options.append(f"max_length={max_length}")
            if is_null is not None:
                field_options.append(f'null={"True" if is_null else "False"}')
            if is_primary_key:
                field_options.append("primary_key=True")
            if field_type == "array":
                field_options.append("default=list")

            model_code += f"    {field_name} = {django_field_type}({', '.join(field_options)})\n"


        # Add empty Meta class before __str__ method
        model_code += "\n"
        model_code += "    class Meta:\n"
        model_code += "        pass\n\n"

        # Add __str__ method with str() typing
        model_code += f"    def __str__(self) -> str:\n"
        model_code += f"        return str(self.UNKNOWN)\n\n"
    
    # for v in infos:
    #     if v in [
    #         "{'$ref': '#/components/schemas/geoPosition'}", 
    #         "{'nullable': True, 'allOf': [{'$ref': '#/components/schemas/geoPoint'}]}", 
    #         "{'type': 'string', 'nullable': True, 'allOf': [{'$ref': '#/components/schemas/geoPoint'}]}"
    #     ]:
    #         print(v)
    #         
    #     
    #     
    # # for v in set(infos):
    # #     print(v)
    # 
    # print()
    # print()
        
    
    return model_code

# Specify the path to the specification file and generate models
spec_file_path = "BrightMLS.RESO.OData v. 1.0.3.json"
spec_data = load_specification(spec_file_path)
django_model_code = generate_django_model_code(spec_data)

# Print or save the generated Django model code
print(django_model_code)

class BuilderModel(BaseModel):
    BuilderModelKey = models.CharField(max_length=255, primary_key=True)
    BuilderModelName = models.CharField(max_length=50, null=True)
    BuilderModelRelatedBuilderModelKey = models.CharField(max_length=255, null=True)
    BuilderModelStatus = models.CharField(max_length=255, null=True)
    BuilderModelCounty = models.CharField(max_length=255, null=True)
    BuilderModelModificationTimestamp = models.DateTimeField(null=True)

    class Meta:
        pass

    def __str__(self) -> str:
        return str(self.UNKNOWN)

class LisBusinessHistory(BaseModel):
    UchPropHistChangeKey = models.CharField(max_length=255)
    UchPropHistListingKey = models.CharField(max_length=255, null=True)
    UchPropHistPartyKey = models.CharField(max_length=255, null=True)
    UchPropHistChangeType = models.CharField(max_length=20, null=True)
    UchPropHistChangeTypePckItemKey = models.CharField(max_length=255, null=True)
    UchPropHistChangeTimestamp = models.DateTime